In [1]:
import pdb
import numpy as np
import os
from tqdm import tqdm_notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


def get_quadrant(point):
    if point[0] >= 0. and point[1] >= 0. :
        return 0
    elif point[0] <= 0. and point[1] >= 0. : 
        return 1
    elif point[0] <= 0. and point[1] <= 0. : 
        return 2
    elif point[0] >= 0. and point[1] <= 0. : 
        return 3
    else :
        raise Exception('invalid input %s', point) 


def passed_origin(x_t, x_t1):
    if get_quadrant(x_t1) == 3 and get_quadrant(x_t) == 0: 
        return True
    else : 
        return False


def fit_quadrant(points, quadrant, desired_amt):
    
    
    points = np.asarray(points)
    slots = []
    slot_size = np.pi / (2 * desired_amt)
    for i in range(int(desired_amt)) : slots.append([])
    if quadrant == 0: 
        points = points[::-1]
    elif quadrant == 1 : 
        points[:, 0] = - points[:, 0]
    elif quadrant == 2 :
        points = points[::-1] 
        points[:, 0] = - points[:, 0]
        points[:, 1] = - points[:, 1]
    elif quadrant == 3 : 
        points[:, 1] = - points[:, 1]

    # import pdb; pdb.set_trace()
    for point in points :
        angle = np.arctan(point[1] / (point[0]+0.000001))
        index = min(int(angle / slot_size), desired_amt - 1)
        slots[int(index)].append(point)

    for i in range(len(slots)):
        if len(slots[i]) == 0 : 
            slots[i] = np.array([0., 0., 0., 0.])
        else :
            full_slot = np.asarray(slots[i])
            slots[i] = full_slot.mean(axis=0)

    points = np.asarray(slots)
    if quadrant == 0: 
        points = points[::-1]
    elif quadrant == 1 : 
        points[:, 0] = - points[:, 0]
    elif quadrant == 2 : 
        points = points[::-1]
        points[:, 0] = - points[:, 0]
        points[:, 1] = - points[:, 1]
    elif quadrant == 3 : 
        points[:, 1] = - points[:, 1]

    return points

def parse_velo(velo):
    # points closer to the origin (0,0,0) are at the end of the point cloud.
    # invert the point cloud such that we begin near the origin. 
    
    # returns: a H x 4 x ? array, split into quadrants
    velo = velo[::-1]
    lines = []
    current_point = velo[0]
    current_quadrant = get_quadrant(current_point)
    current_line = [[], [], [], []]
    quadrant_switches = 0
    for point in velo :
        point_quadrant = get_quadrant(point)
        
        if passed_origin(current_point, point):
            lines.append(current_line)
            current_line = [[], [], [], []]

        current_line[point_quadrant].append(point)
        current_quadrant = point_quadrant
        current_point = point

    return lines


def setmatch(lines,lenLines):
    arr=[[np.array([0,0,0,0]),np.array([0,0,0,0])]]
    if len(lines) > lenLines:
        return lines[:lenLines]
    else:
        for i in range(abs(len(lines)-lenLines)):
            lines.append(arr)
    return lines

def process_velo(velo, points_per_layer, stop=False):
    
    lenLines=29
    lines = parse_velo(velo)
    inverse = quad_to_pc_inv(lines)
    lines = lines[2:-1]
#     print(lines[])
#     print((lines[0]))
#     raise SystemError
#     if(len(lines)!=lenLines):
#         lines=setmatch(lines,lenLines)
    print(len(lines))
    if len(lines) != 29 : raise Exception('invalid nb un of lines')
    out_tensor = np.zeros((29, points_per_layer, 4))
    if stop:
        import pdb; pdb.set_trace()
        x = 1
    for j in range(len(lines)):
        line = lines[j]
#         print(line)
        out_line = np.zeros((points_per_layer, 4))
        for i in range(len(line)):
#             print(type(line[i]))
            if(len(line[i])==0):
                line[i]=[np.array([0.0,0.0,0.0,0.0])]
            gridded = fit_quadrant(line[i], i, points_per_layer / 4)
            out_tensor[j][i*int(points_per_layer/4):(i+1)*int(points_per_layer/4), :] = gridded[::-1]

    return out_tensor, inverse


def quad_to_pc_inv(lines, th=3.):
    # lines is a 63 x 4 array, where each slot has an array of 4d/3d points
    # goal : get an array of points that fills empty spaces
    points = []
    for i in range(len(lines)) :
        line = lines[i] 
        distance = []
        for quad in line : 
            for point in quad : 
                x, y, z = point[:3]
                distance.append(x**2 + y**2)
        distance = np.array(distance)
        std = distance.std()
        sorted_indices = np.argsort(distance)
        median_index = sorted_indices[int(sorted_indices.shape[0]*0.95)]
        median = distance[median_index]

        for quad in line : 
            for point in quad : 
                x, y, z = point[:3]
                dist = x ** 2 + y ** 2 
                if dist < median and (median/dist-1.) > th:#*std : 
                    # blocked point --> scale to get real pt
                    scale = np.sqrt(median / dist)
                    scaled = scale * point
                    points.append(scaled)


    return np.array(points)


In [2]:
def getint(name):
    return int(name.split('.')[0])


In [5]:
# #process whole set of ply files from _output of carla to create npz file to input to parse _velo for matrix creation
# pcd_folder='../../../../carla-0.8.4/PythonClient/_out/'
# folderInOutIndex='0/'
# txt_npy_folder='../../../../carla-0.8.4/PythonClient/_out_npy/'+folderInOutIndex
# os.makedirs(txt_npy_folder)
# files= sorted(os.listdir(pcd_folder),key=getint)
# # print(files)
# for file in files:
#     plyFile=open(pcd_folder+file)
#     newTxtToWrite=open(txt_npy_folder+file.split('.')[0]+'.txt','w')
#     i=0
#     for line in plyFile:
#         if(i<7):
#             i+=1
#         else:
#             line1=line.split(' ')
#             x=float(line1[2])*(-1.0)
#             newTxtToWrite.write(line1[0]+' '+line1[1]+' '+str(x)+'\n')




#process whole set of ply files from _output of carla to create npy file to input to parse _velo for matrix creation
#the code has 2 parts 
# 1--> convert ply files to txt files afte removing the top 7 lines
# 2--> conver the txt files in npy file and save in npt format

# pcd_folder='/home/sabyasachi/Projects/ati/data/data/datasets/Carla/lidarParam1/pair_corrupt/static/'
pcd_folder='/home/sabyasachi/Projects/ati/data/data/datasets/Carla/few_dynamic_runs/110k_dynamic/1/_out/'
# folderInOutIndex='/home/prashant/P/carla-0.8.4/PythonClient/currentDataForCVPR2/lidarParam1/matchPair/dynamicNPY/'
# npy_folder='/home/sabyasachi/Projects/ati/data/data/datasets/Carla/lidarParam1/pair_corrupt/static_NPY/'
npy_folder='/home/sabyasachi/Projects/ati/data/data/datasets/Carla/few_dynamic_runs/110k_dynamic/1_npy/'
# os.makedirs(txt_npy_folder)
files= sorted(os.listdir(pcd_folder),key=getint)
# print(files)
for file in tqdm_notebook(files[:1000]):
    plyFile=open(pcd_folder+file)
    npy=[]
#     newTxtToWrite=open(txt_npy_folder+file.split('.')[0]+'.txt','w')
    i=0
    for line in plyFile:
        if(i<7):
            i+=1
        else:
            point=[]        
            temp=line.split(' ')
            point.append(float(temp[0]))
            point.append(float(temp[1]))
            point.append(float(temp[2]))
            point.append(float(0.00))
            npy.append(point)
    np.array(npy).dump(open(npy_folder+file.split('.')[0]+'.npy', 'wb'))
    print((np.load(npy_folder+file.split('.')[0]+'.npy',allow_pickle=True)).shape)
    
    

(7635, 4)
(7650, 4)
(7789, 4)
(7970, 4)
(8195, 4)
(8514, 4)
(8634, 4)
(8652, 4)
(8636, 4)
(8615, 4)
(8601, 4)
(8585, 4)
(8579, 4)
(8563, 4)
(8563, 4)
(8557, 4)
(8553, 4)
(8550, 4)
(8550, 4)
(8551, 4)
(8548, 4)
(8544, 4)
(8545, 4)
(8547, 4)
(8541, 4)
(8543, 4)
(8532, 4)
(8518, 4)
(8540, 4)
(8510, 4)
(8507, 4)
(8477, 4)
(8438, 4)
(8438, 4)
(8435, 4)
(8392, 4)
(8442, 4)
(8383, 4)
(8356, 4)
(8385, 4)
(8402, 4)
(8381, 4)
(8433, 4)
(8459, 4)
(8449, 4)
(8445, 4)
(8465, 4)
(8483, 4)
(8507, 4)
(8480, 4)
(8505, 4)
(8494, 4)
(8471, 4)
(8426, 4)
(8449, 4)
(8475, 4)
(8424, 4)
(8464, 4)
(8449, 4)
(8444, 4)
(8454, 4)
(8487, 4)
(8505, 4)
(8464, 4)
(8519, 4)
(8524, 4)
(8583, 4)
(8604, 4)
(8587, 4)
(8572, 4)
(8615, 4)
(8671, 4)
(8677, 4)
(8667, 4)
(8657, 4)
(8638, 4)
(8617, 4)
(8572, 4)
(8553, 4)
(8548, 4)
(8574, 4)
(8595, 4)
(8596, 4)
(8580, 4)
(8575, 4)
(8617, 4)
(8640, 4)
(8659, 4)
(8712, 4)
(8715, 4)
(8692, 4)
(8687, 4)
(8641, 4)
(8681, 4)
(8676, 4)
(8645, 4)
(8652, 4)
(8613, 4)
(8577, 4)
(8579, 4)


(8594, 4)
(8596, 4)
(8595, 4)
(8587, 4)
(8589, 4)
(8586, 4)
(8583, 4)
(8580, 4)
(8588, 4)
(8586, 4)
(8585, 4)
(8586, 4)
(8594, 4)
(8595, 4)
(8592, 4)
(8598, 4)
(8602, 4)
(8609, 4)
(8606, 4)
(8608, 4)
(8609, 4)
(8604, 4)
(8605, 4)
(8599, 4)
(8596, 4)
(8595, 4)
(8598, 4)
(8591, 4)
(8584, 4)
(8585, 4)
(8586, 4)
(8594, 4)
(8598, 4)
(8601, 4)
(8599, 4)
(8594, 4)
(8595, 4)
(8605, 4)
(8604, 4)
(8602, 4)
(8611, 4)
(8612, 4)
(8607, 4)
(8609, 4)
(8608, 4)
(8608, 4)
(8615, 4)
(8625, 4)
(8616, 4)
(8610, 4)
(8611, 4)
(8602, 4)
(8614, 4)
(8622, 4)
(8612, 4)
(8609, 4)
(8613, 4)
(8629, 4)
(8631, 4)
(8634, 4)
(8640, 4)
(8637, 4)
(8643, 4)
(8634, 4)
(8628, 4)
(8635, 4)
(8632, 4)
(8642, 4)
(8640, 4)
(8640, 4)
(8642, 4)
(8645, 4)
(8650, 4)
(8633, 4)
(8636, 4)
(8644, 4)
(8643, 4)
(8644, 4)
(8631, 4)
(8638, 4)
(8638, 4)
(8645, 4)
(8641, 4)
(8646, 4)
(8636, 4)
(8633, 4)
(8633, 4)
(8637, 4)
(8643, 4)
(8640, 4)
(8640, 4)
(8635, 4)
(8626, 4)
(8627, 4)
(8640, 4)
(8644, 4)
(8640, 4)
(8636, 4)
(8643, 4)
(8647, 4)


In [ ]:
#  polarCoordMatrix[:,2]
# %reset arrayy
# %xdel testcube
%reset out
#flush all elements with word starting from 'da'
%reset_selective -f [a-z]

In [7]:
# lidarRun='2_npy'
one_run_npy_file=[]
# npyFolder='/home/sabyasachi/Projects/ati/data/data/datasets/Carla/lidarParam1/pair_corrupt/static_NPY/'
npyFolder='/home/sabyasachi/Projects/ati/data/data/datasets/Carla/few_dynamic_runs/110k_dynamic/1_npy/'
files=sorted(os.listdir(npyFolder),key=getint)
print(len(files))

for file in tqdm_notebook(files[0:1000]):
    raw_lidar=np.load(npyFolder+file,allow_pickle=True)
#         print(type(raw_lidar))
#         print(raw_lidar.shape)
    processed_lidar, _ = process_velo(raw_lidar, 512)
#         print(processed_lidar) #(60, 512, 4)
#         print(processed_lidar)
    one_run_npy_file.append(processed_lidar)
print('Completed')
#     np.array(one_run_npy_file).dump(open('_0-5.npy', 'wb'))
    

    
    
#     staticPath='/home/prashant/P/carla-0.8.4/PythonClient/currentDataCVPRprocessed/static/'
#     dynamicPath='/home/prashant/P/carla-0.8.4/PythonClient/currentDataCVPRprocessed/dynamic/'
#     filesStatic  = sorted(os.listdir(staticPath),key=getint)
#     filesDynamic = sorted(os.listdir(staticPath),key=getint)
#     for file in filesStatic:
#         raw_lidar = np.fromfile(staticPath+file, dtype=np.float32).reshape((-1, 4))
#         print((raw_lidar.shape))
        
#         processed_lidar, _ = process_velo(raw_lidar, 512)
#         print(type(processed_lidar)) #(60, 512, 4))
#         print(processed_lidar.shape) #(60, 512, 4)
#         raise SystemError
    
    
    

    

978


29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
2

In [ ]:
# npzFolder  = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/lidarParam1/pair_corrupt/static_NPZ/"
npzFolder  = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/lidarParam1/pair_corrupt/dynamic_NPZ/"
np.savez(npzFolder + '0-2',one_run_npy_file[:2000])
np.savez(npzFolder + '2-4',one_run_npy_file[2000:4000])
np.savez(npzFolder + '4-6',one_run_npy_file[4000:6000])
# np.savez('6-8',one_run_npy_file[6000:8000])
# np.savez('8-10',one_run_npy_file[8000:10000])
# np.savez('4-8',one_run_npy_file[4000:])
# # # np.savez('6-8',one_run_npy_file[6000:8000])
# # # np.savez('8-10',one_run_npy_file[8000:10000])
# # one_run_npy_file.shape
 
  
    
